In [ ]:
import os

# load all datasets in synergy_dataset
path = 'synergy_dataset'
files = os.listdir(path)
files = [f for f in files if f.endswith('.csv')]

In [ ]:
files = ['Leenaars_2020.csv', 'Walker_2018.csv']

In [ ]:
def generate_job(setting, files, model, fe):
  for dataset in files:
    template = f"""apiVersion: batch/v1
kind: Job
metadata:
  name: sim-{dataset.lower().replace("_","-")[:-4]}-{model.lower().replace('_','-')}-{fe.lower().replace('_','-')}
spec:
  ttlSecondsAfterFinished: 100
  template:
      metadata:
        labels:
          app: simulation
      spec:
        restartPolicy: Never
        containers:
        - name: sim
          image: ghcr.io/jteijema/asreview-simulation-project:latest
          resources:
            requests:
              cpu: "1"
          env:
          - name: DATASET
            value: {dataset[:-4]}
          - name: SETTINGS
            value: "{setting}"
          envFrom:
            - secretRef:
                name: exoscale-credentials"""
    # save template to file
    with open(f"jobs/sim-{model}-{fe}-{dataset.lower().replace('_','-')[:-4]}.yaml", "w") as f:
        f.write(template)
    

In [ ]:
# settings generator (no nb)
!mkdir jobs

settings = []

exclude = ["nn-2-layer:tfidf"]

for model in ["logistic", "nn-2-layer", "xgboost", "rf", "svm"]:                                                    # "logistic", "nn-2-layer", "xgboost", "rf", "svm"
    for fe in ["tfidf", "doc2vec", "reuseable_MiniLM", "reuseable_sbert"]:                 # "tfidf", "doc2vec", ""reuseable_MiniLM", "reuseable_sbert"
                if not f"{model}:{fe}" in exclude:
                    setting = "-m {model} -e {fe}".format(model=model, fe=fe)
                    settings.append(setting)
                    generate_job(setting, files, model, fe)

In [ ]:
# get all jobs
jobs = os.listdir('jobs')
for job in jobs:
    !kubectl create -f ./jobs/{job}

In [ ]:
!kubectl get pods